In [3]:
import numpy as np

In [5]:
X_train = np.load('Fashion_MNIST/X_train.npy')
X_test = np.load('Fashion_MNIST/X_test.npy')
y_train = np.load('Fashion_MNIST/y_train.npy')
y_test = np.load('Fashion_MNIST/y_test.npy')

In [6]:
X_train.shape

(60000, 784)

In [7]:
y_train.shape

(60000,)

In [8]:
X_test.shape

(10000, 784)

In [9]:
y_test.shape

(10000,)

In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
encoder = OneHotEncoder(sparse=False)

In [12]:
encoder.fit(y_train.reshape(-1, 1))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [13]:
y_train = encoder.transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

In [14]:
y_train.shape

(60000, 10)

In [15]:
def get_next_batch(X, y, size):
    m = X.shape[0]
    index = np.random.permutation(np.arange(m))
    for i in range(m//size):
        batch_i = index[i*size: i*size+size]
        yield X[batch_i], y[batch_i]

Conv k=3x3, s=1x1, p=same

MaxPool k=2x2, s=2x2 

``` 
X -> Conv16 -> Maxpool -> Conv32 -> Conv64 -> Maxpool ->Flattening -> fc128 -> fc64 -> y_hat10

X -> --W1-- -> ------- -> --W2-- -> --W3-- -> ------- ->Flattening -> --W4- -> -W5- -> --W6---
```

In [ ]:
import tensorflow as tf

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

In [ ]:
X = tf.reshape(X, [None, 28, 28, 1])

In [ ]:
# None, 28, 28, 1
W1 = tf.Variable(tf.random_normal(shape=[3, 3, 1, 16]))
b1 = tf.Variable(tf.random_normal(shape=[16]))
# None, 28, 28, 1
# Pool -> None, 14, 14, 16
W2 = tf.Variable(tf.random_normal(shape=[3, 3, 16, 32]))
b2 = tf.Variable(tf.random_normal(shape=[32]))
# None, 14, 14, 32
W3 = tf.Variable(tf.random_normal(shape=[3, 3, 32, 64]))
b3 = tf.Variable(tf.random_normal(shape=[64]))
# None, 14, 14, 64
# Pool -> None, 7, 7, 64
W4 = tf.Variable(tf.random_normal(shape=[7*7*64, 128]))
b4 = tf.Variable(tf.random_normal(shape=[128]))

W5 = tf.Variable(tf.random_normal(shape=[128, 64]))
b5 = tf.Variable(tf.random_normal(shape=[64]))

W6 = tf.Variable(tf.random_normal(shape=[64, 10]))
b6 = tf.Variable(tf.random_normal(shape=[10]))

In [ ]:
Z1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding="SAME") + b1
A1 = tf.nn.relu(Z1)
pool1 = tf.nn.max_pool(A1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

Z2 = tf.nn.conv2d(pool1, W2, strides=[1, 1, 1, 1], padding="SAME") + b2
A2 = tf.nn.relu(Z2)
Z3 = tf.nn.conv2d(A2, W3, strides=[1, 1, 1, 1], padding="SAME") + b3
A3 = tf.nn.relu(Z3)

pool3 = tf.nn.max_pool(A3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

conv_out = tf.reshape(pool3, [None, 7*7*64])
Z4 = tf.matmul(W4, conv_out) + b4
A4 = tf.nn.relu(Z4)

Z5 = tf.matmul(W5, A4) + b5
A5 = tf.nn.relu(Z5)

logits = tf.matmul(W5, A5) + b5 #The output of the network before the last activation is called Logits

In [ ]:
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y)
cost = tf.reduce_mean(loss)

train_step = tf.train.AdamOptimizer(0.001).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1, 101):
        
        for batch in get_next_batch(X_train, y_train, 64):
            sess.run(train_step, {X:batch[0], y:batch[1]})
            
        if epoch % 10 == 0:
            test_acc, test_loss = sess.run([accuracy, cost], {X:X_test, y:y_test})
            
            batch = get_next_batch(X_train, y_train, 1000).__next__() #just getting 1000 random data from Train
            train_acc, train_loss = sess.run([accuracy, cost], {X:batch[0], y:batch[1]})
            print("Epoch", epoch, "Train Acc:", np.round(train_acc, 5), \
                  "Train Loss:", np.round(train_loss, 5), \
                  "Test Acc:", np.round(test_acc, 5), "Test Loss:", np.round(test_loss, 5))